#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [ ]:
!pip install folium
!pip install scikit-learn

Распакуем наши данные из архива.

In [ ]:
!unzip ...

unzip:  cannot find or open ..., ....zip or ....ZIP.


Загрузим данные из csv файла в датафрейм.

In [ ]:
df = pd.read_csv('/input_data.csv', sep=';')
display(df)

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0


Отобразим на карте координаты наших построек.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1500]

m = folium.Map(location=[55.7512, 37.6184], zoom_start=10)


lats = map_df['geo_lat'].loc[:1500]
longs = map_df['geo_lon'].loc[:1500]

for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.3

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi / 2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2.0)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def is_within_radius(lat, lon, center_lat, center_lon, radius_km):
    return haversine_distance(lat, lon, center_lat, center_lon) <= radius_km

moscow_center = (55.7558, 37.6176)
saint_petersburg_center = (59.9343, 30.3351)

df['is_Moscow'] = df.apply(lambda row: is_within_radius(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1], 20), axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: is_within_radius(row['geo_lat'], row['geo_lon'], saint_petersburg_center[0], saint_petersburg_center[1], 20), axis=1)

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [ ]:
columns_to_remove = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']
df = df.drop(columns=columns_to_remove)

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: date, building_type, object_type, postal_code, street_id, id_region, house_id

Числовые: price, level, levels, rooms, area, kitchen_area, geo_lat, geo_lon

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
categorical_columns = ['date', 'building_type']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_features = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))
final_df = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [ ]:
df['date'] = pd.to_datetime(df['date'])

min_date = df['date'].min()
df['days_since_first'] = (df['date'] - min_date).dt.days

df['levels'] = df['levels'].replace(0, 1)
df['floor_ratio'] = df['level'] / df['levels']

df.drop(columns=['date'], inplace=True)

scaler = StandardScaler()

numerical_features = ['days_since_first', 'floor_ratio', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon']
df[numerical_features] = scaler.fit_transform(df[numerical_features])


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
      self.n_neighbors = n_neighbors
      self.metric = metric
      self.X_train = None
      self.y_train = None

    def fit(self, X, y):
      self.X_train = np.array(X)
      self.y_train = np.array(y)

    def predict(self, X):
      X = np.array(X)
      distances = pairwise_distances(X, self.X_train, metric=self.metric)
      neighbors_indices = np.argsort(distances, axis=1)[:, :self.n_neighbors]
      predictions = np.array([self.y_train[neighbors].mean() for neighbors in neighbors_indices])
      return predictions

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None
        self.accumulated_squared_grad = None
    def fit(self, X, y):

        X = X.values if isinstance(X, (pd.DataFrame, pd.Series)) else X
        y = y.values if isinstance(y, (pd.DataFrame, pd.Series)) else y

        if not isinstance(X, np.ndarray):
            X = X.to_numpy()
        X = X.astype(np.float64)

        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        if self.optimization == 'Momentum':
            self.velocity = np.zeros(n_features)
        elif self.optimization == 'AdaGrad':
            self.accumulated_squared_grad = np.zeros(n_features)

        for iter in range(self.max_iter):
            for i in range(n_samples):
                y_predicted = np.dot(X[i], self.weights) + self.bias
                error = y_predicted - y[i]
                dw = (2 * error * X[i])
                db = 2 * error

                if self.optimization == 'SGD':
                    self.weights -= self.learning_rate * dw
                    self.bias -= self.learning_rate * db
                elif self.optimization == 'Momentum':
                    self.velocity = self.decay_rate * self.velocity + (1 - self.decay_rate) * dw
                    self.weights -= self.learning_rate * self.velocity
                    self.bias -= self.learning_rate * db
                elif self.optimization == 'AdaGrad':
                    self.accumulated_squared_grad += dw**2
                    adjusted_lr = self.learning_rate / (np.sqrt(self.accumulated_squared_grad) + self.epsilon)
                    self.weights -= adjusted_lr * dw
                    self.bias -= adjusted_lr * db

    def predict(self, X):
        X = X.values if isinstance(X, (pd.DataFrame, pd.Series)) else X
        if not isinstance(X, np.ndarray):
          X = X.to_numpy()
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
import numpy as np

X = df.drop(columns=['price'])
y = df['price']

X = X.head(1000)
y = y.head(1000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_regressor = KNNRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)
knn_pred = knn_regressor.predict(X_test)

sklearn_knn = KNeighborsRegressor(n_neighbors=5)
sklearn_knn.fit(X_train, y_train)
sklearn_knn_pred = sklearn_knn.predict(X_test)

linear_regressor = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
linear_regressor.fit(X_train, y_train)
linear_pred = linear_regressor.predict(X_test)

sklearn_lr = SklearnLinearRegression()
sklearn_lr.fit(X_train, y_train)
sklearn_lr_pred = sklearn_lr.predict(X_test)

def calculate_errors(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

knn_mse, knn_mae, knn_rmse = calculate_errors(y_test, knn_pred)
linear_mse, linear_mae, linear_rmse = calculate_errors(y_test, linear_pred)

sklearn_knn_mse, sklearn_knn_mae, sklearn_knn_rmse = calculate_errors(y_test, sklearn_knn_pred)
sklearn_linear_mse, sklearn_linear_mae, sklearn_linear_rmse = calculate_errors(y_test, sklearn_lr_pred)

print("Custom KNN Regressor:")
print(f"MSE: {knn_mse}, MAE: {knn_mae}, RMSE: {knn_rmse}")

print("\nSklearn KNN Regressor:")
print(f"MSE: {sklearn_knn_mse}, MAE: {sklearn_knn_mae}, RMSE: {sklearn_knn_rmse}")

print("\nCustom Linear Regression:")
print(f"MSE: {linear_mse}, MAE: {linear_mae}, RMSE: {linear_rmse}")

print("\nSklearn Linear Regression:")
print(f"MSE: {sklearn_linear_mse}, MAE: {sklearn_linear_mae}, RMSE: {sklearn_linear_rmse}")

Custom KNN Regressor:
MSE: 3.2125378684813604e-05, MAE: 0.002002653523634339, RMSE: 0.005667925430421047

Sklearn KNN Regressor:
MSE: 3.212873690989191e-05, MAE: 0.002003297821330714, RMSE: 0.005668221670849853

Custom Linear Regression:
MSE: 3.0755872799147164e-05, MAE: 0.0026975800163709473, RMSE: 0.005545797760390038

Sklearn Linear Regression:
MSE: 2.914874135815849e-05, MAE: 0.0027134456201237745, RMSE: 0.005398957432519587
